의존성 및 키 세팅

In [12]:
%%capture --no-stderr
%pip install langchain langchainhub langchain-openai langchain-openai langchain_chroma langchain-text-splitters langchain_community

In [13]:
import os

key_path = '../openai_key.txt'
with open(key_path, 'r') as file:
    os.environ["OPENAI_API_KEY"] = file.read().strip()

In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

메세지 생성

In [31]:
# 프롬프트를 가져오거나 등록할 수 있음
from langchain import hub

# https://smith.langchain.com/hub/rlm/rag-prompt
prompt = hub.pull("rlm/rag-prompt")

# 프롬프트 템플릿 채우는 예시
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

print(prompt)
print(example_messages)


input_variables=['context', 'question'] metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]
[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]


In [18]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [27]:
# 청크 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# 설정된 청크로 분리
splits = text_splitter.split_documents(docs)
# 청크된 문서로 Chroma 객체 생성
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

In [37]:
# 1. 3개의 블로그 포스팅 본문을 Load: WebBaseLoader 활용
# https://python.langchain.com/v0.2/docs/integrations/document_loaders/web_base/

# 동시에 여러 웹사이트에서 문서를 로드할 수 있음
%pip install --upgrade --quiet  nest_asyncio
import nest_asyncio

nest_asyncio.apply()

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

loader = WebBaseLoader(urls)
loader.requests_per_second = 1
documents = loader.aload()

print(docs[0].page_content[:100])


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Fetching pages: 100%|##########| 3/3 [00:00<00:00, 27.21it/s]







LLM Powered Autonomous Agents | Lil'Log







































Lil'Log










In [39]:
# 2. 불러온 본문을 Split (Chunking) : recursive text splitter 활용
!pip install faiss-cpu
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.split_documents(documents)


  Obtaining dependency information for faiss-cpu from https://files.pythonhosted.org/packages/c2/2a/e72ebe364754fb110cfd802aec92c2a1bb3590f55d6c1910b3c6fe3c6d18/faiss_cpu-1.8.0.post1-cp39-cp39-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 21.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [111]:
# 3. Chunks 를 임베딩하여 Vector store 저장: openai, chroma 사용
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vectorstore = FAISS.from_documents(texts, embeddings)

retriever = vectorstore.as_retriever()

In [110]:
query = "agent memory"
docs = retriever.invoke(query)
print(len(docs))
print(docs[0].page_content)

4
page_content='Agent System Overview#' metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en'}


In [53]:
# Retrieve more documents with higher diversity
# Useful if your dataset has many similar documents
retriever1 = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 6, 'lambda_mult': 0.25}
)
print(retriever1.invoke(query))

# Fetch more documents for the MMR algorithm to consider
# But only return the top 5
retriever2 = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 5, 'fetch_k': 50}
)
print(retriever2.invoke(query))

# Only retrieve documents that have a relevance score
# Above a certain threshold
retriever3 = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.8}
)
print(retriever3.invoke(query))

# Only get the single most similar document from the dataset
retriever4 = vectorstore.as_retriever(search_kwargs={'k': 1})
print(retriever4.invoke(query))

# Use a filter to only retrieve documents from a specific paper
retriever5 = vectorstore.as_retriever(
    search_kwargs={'filter': {'paper_title': 'GPT-4 Technical Report'}}
)
print(retriever5.invoke(query))


[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en'}, page_content='Agent System Overview#'), Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, 

/Users/whale/work/whale.so/venv/lib/python3.9/site-packages/langchain_core/vectorstores/base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


[]
[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en'}, page_content='Agent System Overview#')]
[]


In [128]:
# 5. User query와 retrieved chunk 에 대해 relevance 가 있는지를 평가하는 시스템 프롬프트 작성: retrieval 퀄리티를 LLM 이 스스로 평가하도록 하고, 관련이 있으면 {‘relevance’: ‘yes’} 관련이 없으면 {‘relevance’: ‘no’} 라고 출력하도록 함. ( JsonOutputParser() 를 활용 )

from langchain import hub

prompt = hub.pull("rlm/rag-document-relevance")
prompt

StructuredPrompt(input_variables=['input'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-document-relevance', 'lc_hub_commit_hash': '123af323ca4720fc195f1d6966ba25fa67348c351972be2088faf2b22f2de056'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a grader assessing relevance of a retrieved document to a user question.\n\nIf the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \\n\n\nIt does not need to be a stringent test. The goal is to filter out erroneous retrievals. \\n\n\nGive a binary score 1 or 0 score, where 1 means that the document is relevant to the question.', template_format='mustache')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='Retrieved documents: {{input.documents}} \n\nUser question: {{input.question}}', template_format='mustache'))], schema_={'type': 'object', 'title': 'Score_criteria', 'required': ['Score', 'Expl

In [105]:
format_instructions = prompt.invoke(
    {"input": {
        "question": query,
        "documents": docs[0]
    }}
).to_messages()

format_instructions

[SystemMessage(content='You are a grader assessing relevance of a retrieved document to a user question.\n\nIf the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \\n\n\nIt does not need to be a stringent test. The goal is to filter out erroneous retrievals. \\n\n\nGive a binary score 1 or 0 score, where 1 means that the document is relevant to the question.'),
 HumanMessage(content="Retrieved documents: page_content='Agent System Overview#' metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': &quot;LLM Powered Autonomous Agents | Lil'Log&quot;, 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solv

In [130]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

joke_query = "Tell me a joke."

parser = JsonOutputParser()

chain = prompt | llm | parser

chain = (
        {"input": {"question": RunnablePassthrough(),"documents": retriever | format_docs}}
        | prompt
        | llm
        | StrOutputParser()
)

chain.invoke(joke_query)

'Score: 0'